In [1]:
import pandas as pd
from requests_html import HTMLSession

session = HTMLSession()

In [83]:
def get_user_from_tr(tr):
    display_name_block = tr.find('.display_name', first=True)
    if not display_name_block: return
    
    display_name = display_name_block.text

    social_links = tr.find('.social', first=True).links
    get_kv = lambda link: (link.split('.com')[0].replace('https://',''), link)
    social_dict = dict([ get_kv(link) for link in social_links])

    twitter  = social_dict['twitter'] if 'twitter' in social_dict else None
    twitter_bio_block = session.get(twitter).html.find('.note', first=True) if twitter else None
    twitter_bio = twitter_bio_block.text if twitter_bio_block else None

    user = {
        'diaplay_name': display_name,
        **social_dict,
        'twitter_bio': twitter_bio,
    }
    return user

In [84]:
user_list = []
for base_url in [
    f"https://connpass.com/event/124747/ptype/174893/waitlist/",
    f"https://connpass.com/event/124747/ptype/174893/participants/",
    f"https://connpass.com/event/124747/ptype/174893/cancelled/",
]:
    for page in range(1, 114514):
        url = f"{base_url}?page={page}"
        print(url)
        r = session.get(url)

        users = map(get_user_from_tr, r.html.find('#main tr'))
        users = filter(bool, users)
        user_list += list(users)

        paging_area = r.html.find('.paging_area', first=True)
        if not ( paging_area and '次へ' in paging_area.text ) : break

https://connpass.com/event/124747/ptype/174893/waitlist/?page=1
https://connpass.com/event/124747/ptype/174893/waitlist/?page=2
https://connpass.com/event/124747/ptype/174893/waitlist/?page=3
https://connpass.com/event/124747/ptype/174893/participants/?page=1
https://connpass.com/event/124747/ptype/174893/cancelled/?page=1


In [85]:
df = pd.DataFrame(user_list)

In [87]:
df.to_csv('プランクトンサミット#3.csv')

In [96]:
gsheet_url = "https://docs.google.com/spreadsheets/d/1cGpZ6aDMdCXeefIbqAG-MHz1_CjrEuU3Qd1CcN8qpFo/edit#gid=1383004683"
csv_export_url = gsheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
pd.read_csv(
    csv_export_url,
)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2
